In [1]:
import numpy as np 
import pandas as pd 

import glob, pylab
import pydicom
from os import listdir
from os.path import isfile, join
import matplotlib.pylab as plt
import os
import seaborn as sns
import cv2
import tensorflow as tf


In [3]:
# Data directory
DATA_DIR = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'

In [4]:
# Labels for the images in the training data
train_df = pd.read_csv(DATA_DIR + 'stage_2_train.csv')
train_df.head()

,ID,Label
0,ID_12cadc6af_epidural,0
1,ID_12cadc6af_intraparenchymal,0
2,ID_12cadc6af_intraventricular,0
3,ID_12cadc6af_subarachnoid,0
4,ID_12cadc6af_subdural,0


In [ ]:
# Check missing values
train_df.isna().sum()

In [33]:
# Train image directory
train_img_dir = DATA_DIR + 'stage_2_train/'
test_img_dir = DATA_DIR + 'stage_2_test/'
train_images = os.listdir(train_img_dir)
test_images = os.listdir(test_img_dir)

In [ ]:
train_images[:10]

In [ ]:
test_images[:10]

In [ ]:
fig=plt.figure(figsize=(15, 10))
columns = 5
rows = 4
for i in range(1, columns*rows +1):
    img = pydicom.read_file(train_img_dir + train_images[i])
    fig.add_subplot(rows, columns, i)
    plt.imshow(img.pixel_array, cmap=plt.cm.bone)
    fig.add_subplot

In [ ]:
# DICOM format comes with metadata
print(img)

Let's create a new data frame where we have one row for each patient ID and one column per diagnosis. 

In [5]:
# Get class names
CLASS_NAMES = [id.split('_')[2] for id in train_df[:6]['ID'].values]
print(CLASS_NAMES)

['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']


In [ ]:
train_df['id'] = train_df['ID'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
train_df['class'] = train_df['ID'].apply(lambda x: x.split('_')[2])
train_df.head()

In [ ]:
new_train_df = train_df.drop(columns = ['ID'])
new_train_df = new_train_df.drop_duplicates()

In [ ]:
# Create one row per image ID
CLASS_NAMES = np.array(CLASS_NAMES)
grouped = new_train_df.groupby(by='id')
labels = []
ids = []
classes = []
for name, group in grouped:
    #label = [item[0] for item in group.values]
    label = list(group['Label'].values)
    labels.append(label)
    classes.append(''.join(list(CLASS_NAMES[np.array(label) == 1])))
    ids.append(name)

train_id_df = pd.DataFrame(labels, columns = CLASS_NAMES)
train_id_df['ID'] = ids
train_id_df['class'] = classes
train_id_df = train_id_df.sample(frac=1)
train_id_df.head()

In [ ]:
# Write the new dataframe to csv
train_id_df.to_csv('one_hot_labels.csv', index = False)

In [ ]:
# Check class frequencies
grouped_by_class = new_train_df.groupby('class').sum()
sns.barplot(y=grouped_by_class.index, x=grouped_by_class.Label, palette="deep")

In [ ]:
fig=plt.figure(figsize=(10, 8))
sns.countplot(x="class", hue="Label", data=new_train_df)

Labels are highly imbalanced so we will have to do something about this. 

# Windows

CT images uses radiodensity instead of gray intesity and the units for the radiodensity is called Hounsfield units (HU). The usual interval for a CT scan is between -1000 and 1000 HUs. If we directly turn this interval into 0-255 units of shade of gray, each unit would correspond to 8 HUs. Human eye can detect about ~16 shades of gray which corresponds to ~130 HUs. Most of the changes due to a hemorrhage happens in an interval of 80-100 HUs, so in order to able to detect these changes we should use a smaller window than -1000 to 1000. 

In [50]:
def get_first_of_dicom_field_as_int(x):
    #get x[0] as int if x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value] #window width
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

def window_image(dcm, window_center, window_width, correction = True):
    if correction:
        if (dcm.BitsStored == 12) & (dcm.PixelRepresentation == 0) & (int(dcm.RescaleIntercept) > -100):
            correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img = np.clip(img, img_min, img_max)
    
    img = (img - img_min) / (img_max - img_min)
                    
    return img
                    
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x >= px_mode] = x[x >= px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000


def create_window_channels(img_path):
    dcm = pydicom.read_file(img_path)
    window_center, window_width = get_windowing(dcm)
                    
    brain_channel = window_image(dcm, 40, 80)
    subdural_channel = window_image(dcm, 80, 200)
    default_channel = window_image(dcm, window_center, window_width)
    
    multi_channel = np.array([brain_channel, subdural_channel, default_channel]).transpose(1,2,0)
    return multi_channel

In [ ]:
random_idx = np.random.randint(0,7000, size = 1000)
window_centers = []
window_widths = []
for img_id in np.array(train_images)[random_idx]:
    dcm = pydicom.read_file(train_img_dir + img_id)
    window_centers += [get_first_of_dicom_field_as_int(dcm[('0028', '1050')].value)]
    window_widths += [get_first_of_dicom_field_as_int(dcm[('0028', '1051')].value)]

In [ ]:
sns.countplot(window_widths)

In [ ]:
sns.countplot(window_centers)

Let's plot an example image with the corresponding diagnosis. 

In [7]:
ground_truth = pd.read_csv("../input/rsna-one-hot-labels/one_hot_labels.csv")
ground_truth = ground_truth.fillna('x')
ground_truth['one_hot'] = list(ground_truth[CLASS_NAMES].values)
ground_truth.head()

,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,ID,class,one_hot
0,0,0,0,0,0,0,ID_327e33972,x,"[0, 0, 0, 0, 0, 0]"
1,0,0,1,0,0,1,ID_cedd11e21,intraventricularany,"[0, 0, 1, 0, 0, 1]"
2,0,0,0,0,0,0,ID_f88929622,x,"[0, 0, 0, 0, 0, 0]"
3,0,0,0,0,0,0,ID_02595b48e,x,"[0, 0, 0, 0, 0, 0]"
4,0,0,1,0,0,1,ID_769385a56,intraventricularany,"[0, 0, 1, 0, 0, 1]"


In [ ]:
def show_image(img_name):
    arr = create_window_channels(os.path.join(train_img_dir,img_name + '.dcm'))
    channels = ['brain', 'subdural', 'default', 'all']
    fig=plt.figure(figsize=(20, 60))

    for i in range(3):
        plt.subplot(1,4,i+1)
        plt.imshow(arr[:,:,i], cmap=plt.cm.bone)
        plt.title(channels[i])
    
    plt.subplot(1,4,4)
    plt.imshow(arr, cmap=plt.cm.bone) 
    plt.title(channels[3])
    

In [ ]:
show_image(train_images[0].split('.')[0])

# Create tfrecords

In [8]:
from sklearn.model_selection import train_test_split

train_df, valid_df, train_idx, valid_idx = train_test_split(ground_truth, ground_truth['class'], stratify = ground_truth['class'], test_size = 0.05, random_state = 42)

In [45]:
def get_base_model():
    input_shape = (224,224,3)
    base_model = tf.keras.applications.DenseNet201(include_top = False,
                                                   weights = 'imagenet',
                                                   input_shape = input_shape,
                                                   pooling = 'avg')
    preprocess = tf.keras.applications.densenet.preprocess_input
    
    return base_model, preprocess, input_shape

In [40]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value = [value]))

def create_example_record(one_hot_label, bottleneck_features):
    feature = {'one_hot_label' : _bytes_feature(one_hot_label),
              'bottleneck_features' : _bytes_feature(bottleneck_features)}
    example = tf.train.Example(features = tf.train.Features(feature = feature))
    return example.SerializeToString()


In [10]:
def create_tfrecords(df, name = None):
    record_dir = 'TFRecords'
    
    if not os.path.isdir(record_dir):
        os.mkdir(record_dir)
        
    filename = record_dir + '/' + name + '.tfrecords'
    writer = tf.io.TFRecordWriter(filename)
    count = 0
    
    for row in df.itertuples():
        img_path = DATA_DIR + 'stage_2_train/' + row.ID + '.dcm'
        try:
            img = create_window_channels(img_path)
        except:
            continue
        #img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LINEAR)
        img = tf.image.resize(img, size = (224,224))
        img = tf.expand_dims(img, axis = 0)
        
        img = preprocess(img)
        bottleneck_features = base_model.predict(img)
        
        bottleneck_features = tf.io.serialize_tensor(bottleneck_features)
        one_hot_label = tf.io.serialize_tensor(row.one_hot)
                
        writer.write(create_example_record(one_hot_label, bottleneck_features))
        
        count += 1
        if count % 1000 == 0:
            print(count)
    writer.close()

In [46]:
base_model, preprocess, input_shape = get_base_model()

74842112/74836368 [==============================] - 2s 0us/step


In [ ]:
create_tfrecords(train_df[650000:], name = 'train_650000_')

In [ ]:
create_tfrecords(valid_df, name = 'valid')

In [12]:
def read_tfrecord(example):
    features = {
                'one_hot_label': tf.io.FixedLenFeature((), tf.string),
                'bottleneck_features': tf.io.FixedLenFeature((), tf.string)
    }
    example = tf.io.parse_single_example(example, features)
    bottleneck_features = tf.io.parse_tensor(example['bottleneck_features'], out_type = tf.float32)
    bottleneck_features = tf.reshape(bottleneck_features, [-1])
    one_hot_label = tf.io.parse_tensor(example['one_hot_label'], out_type = tf.int64)
    
    bottleneck_features.set_shape([1920])
    one_hot_label.set_shape([6])
    
    return bottleneck_features, one_hot_label

def create_dataset(record_dir, batch_size = 32):
    #train_dataset = tf.data.Dataset.list_files(record_dir + "train*.tfrecords")
    filenames = tf.io.gfile.glob(record_dir + "train*.tfrecords")
    train_dataset = tf.data.TFRecordDataset(filenames)
    train_dataset = train_dataset.map(read_tfrecord).repeat().shuffle(10000).batch(batch_size)
    
    #val_dataset = tf.data.Dataset.list_files(record_dir + "valid*.tfrecords")
    val_filenames = tf.io.gfile.glob(record_dir + "valid*.tfrecords")
    val_dataset = tf.data.TFRecordDataset(val_filenames)
    val_dataset = val_dataset.map(read_tfrecord).batch(batch_size)
    
    return train_dataset, val_dataset

## Get class weights

Create class weights to use in the loss function. The classes with not enough representation will get higher weight. So the model will pay more attention to mistakes in these classes. 

In [14]:
ground_truth.head()

,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,ID,class,one_hot
0,0,0,0,0,0,0,ID_327e33972,x,"[0, 0, 0, 0, 0, 0]"
1,0,0,1,0,0,1,ID_cedd11e21,intraventricularany,"[0, 0, 1, 0, 0, 1]"
2,0,0,0,0,0,0,ID_f88929622,x,"[0, 0, 0, 0, 0, 0]"
3,0,0,0,0,0,0,ID_02595b48e,x,"[0, 0, 0, 0, 0, 0]"
4,0,0,1,0,0,1,ID_769385a56,intraventricularany,"[0, 0, 1, 0, 0, 1]"


In [15]:
length = len(ground_truth)
class_weights = [length/(8*ground_truth[label].sum()) for label in CLASS_NAMES[:-1]]

In [16]:
class_weights += [sum(class_weights)]

In [17]:
class_weights

[29.9206279809221,
 2.605359516030788,
 3.5909320740316732,
 2.6377119831814997,
 1.9950891532035788,
 40.74972070736963]

## Create a model

In [18]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(2048, input_shape = (1920,), activation = 'relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(6, activation = 'sigmoid'))
    
    return model

In [19]:
class WeightedLoss(tf.keras.losses.Loss):
    def __init__(self, class_weights, **kwargs):
        self.class_weights = class_weights
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        y_true = tf.expand_dims(y_true, -1)
        y_pred = tf.expand_dims(y_pred, -1)

        loss_per_class = tf.keras.losses.BinaryCrossentropy(reduction = tf.keras.losses.Reduction.NONE)
        
        weighted_average = tf.reduce_sum(loss_per_class(y_true, y_pred) * self.class_weights, axis = 1)/tf.reduce_sum(self.class_weights)

        return tf.reduce_mean(weighted_average)
                        
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "class_weights": self.class_weights}

In [20]:
batch_size = 32
num_steps = len(train_df) // batch_size
epochs = 20
learning_rate = 0.001

In [21]:
tf.keras.backend.clear_session()

model = get_model()

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                        loss = WeightedLoss(class_weights))

In [22]:
record_dir = "../input/rsnabottleneckvalid150000/"
train_dataset, val_dataset = create_dataset(record_dir, batch_size = batch_size)

In [23]:
scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                 factor=0.9, 
                                                 patience=2, 
                                                 min_delta=0.0001, 
                                                 cooldown=0, 
                                                 min_lr=0.0001)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = 'model_at_epoch_{epoch:02d}.hdf5', 
                                                monitor='val_loss', 
                                                save_best_only=True)

model.fit(train_dataset, 
          epochs = epochs, 
          steps_per_epoch = num_steps,
          validation_data = val_dataset,
          callbacks = [scheduler, checkpoint])

Train for 22348 steps
Epoch 1/20
22348/22348 [==============================] - 132s 6ms/step - loss: 0.1966 - val_loss: 0.1856
Epoch 2/20
22348/22348 [==============================] - 125s 6ms/step - loss: 0.1881 - val_loss: 0.1836
Epoch 3/20
22348/22348 [==============================] - 124s 6ms/step - loss: 0.1861 - val_loss: 0.1810
Epoch 4/20
22348/22348 [==============================] - 121s 5ms/step - loss: 0.1852 - val_loss: 0.1798
Epoch 5/20
22348/22348 [==============================] - 120s 5ms/step - loss: 0.1846 - val_loss: 0.1803
Epoch 6/20
22348/22348 [==============================] - 119s 5ms/step - loss: 0.1842 - val_loss: 0.1804
Epoch 7/20
22348/22348 [==============================] - 120s 5ms/step - loss: 0.1835 - val_loss: 0.1798
Epoch 8/20
22348/22348 [==============================] - 120s 5ms/step - loss: 0.1835 - val_loss: 0.1789
Epoch 9/20
22348/22348 [==============================] - 120s 5ms/step - loss: 0.1830 - val_loss: 0.1791
Epoch 10/20
22348/22348 

In [65]:
submission_df = pd.read_csv(DATA_DIR + 'stage_2_sample_submission.csv', index_col = 'ID')

In [66]:
for img_id in test_images:
    img_path = test_img_dir + img_id
    img = create_window_channels(img_path)
    
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LINEAR)
    img = np.expand_dims(img, axis = 0)

    img = preprocess(img)
    bottleneck_features = base_model.predict(img)
    
    prediction = model.predict(bottleneck_features)
    id_type_names = [img_id.split('.')[0] + '_' + cls for cls in CLASS_NAMES]
    submission_df.loc[id_type_names, 'Label'] = prediction


  0%|          | 13/121232 [00:02<5:26:25,  6.19it/s]


KeyboardInterrupt: 

In [28]:
submission_df.head(10)

,Label
ID,
ID_0fbf6a978_epidural,0.5
ID_0fbf6a978_intraparenchymal,0.5
ID_0fbf6a978_intraventricular,0.5
ID_0fbf6a978_subarachnoid,0.5
ID_0fbf6a978_subdural,0.5
ID_0fbf6a978_any,0.5
ID_d62ec3412_epidural,0.5
ID_d62ec3412_intraparenchymal,0.5
ID_d62ec3412_intraventricular,0.5


In [ ]:
submission_df.to_csv('submission.csv')